In [ ]:
!pip install transformers
!pip install ratsnlp

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import torch
import transformers
from transformers import AutoModelWithLMHead, PreTrainedTokenizerFast
from transformers import GPT2LMHeadModel

import re
import random
import os

import pandas as pd
import numpy as np
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
name_list=['컨츄리와인_영동포도와인','산내들_소백산스위트와인','금이산농원_복숭아와인','배상면주가_빙탄복','덕유양조_산머루주',
           '느린마을 막걸리','술 리뷰(곰표막걸리)','술 리뷰(복순도가)','술 리뷰(죽향도가)',
           '감홍로 40도','금설 35도','더한 매실원주 13도','도원결의 15도','박재서 명인 안동 소주 22도','박재서 명인 안동 소주 35도',
           '박재서 명인 안동 소주 45도','서울의밤 25도','안동소주 일품 21도','안동소주 일품 40도',
           '양촌양조 무농약 우렁이쌀','청주_고헌정 동학 1957 특선','청주_오메기맑은술']

In [ ]:
data = pd.DataFrame()

for name in name_list:
  df = pd.read_csv(f"/content/drive/MyDrive/술/{name}.csv", encoding='utf-8-sig')
  df = df.iloc[:,:3]
  df.dropna(inplace=True)
  df = df[df.text.str.contains('맛|향|달|넘김|부드|단|도수|냄새')]
  df = df[~df.text.str.contains('배송|포장')]
  df = df[df['text'].str.len()>30].sort_values(by='text',ascending=False).head(600)
  df.text = f"{name} " + df.text
  data = pd.concat([data,df])
  print(len(df))
  
data.drop_duplicates(['text'],ignore_index=True, inplace=True) # 중복 제거
data.columns = ['date', 'star','doc']
data.doc = data.doc.str.replace("[^가-힣 ]"," ") # 한글만 남기고 특수문자, 숫자, 영어 삭제
data.doc = data.doc.str.replace("\s+", " ") # white space 삭제
data.doc = data.doc.str.replace("럽고목","럽고 목")



218
600
600
600
165
600
600
600
600
129
173
583
412
600
525
600
600
175
385
78
172
359


<ipython-input-4-88b2eb294516>:16: FutureWarning: The default value of regex will change from True to False in a future version.
  data.doc = data.doc.str.replace("[^가-힣 ]"," ") # 한글만 남기고 특수문자, 숫자, 영어 삭제
<ipython-input-4-88b2eb294516>:17: FutureWarning: The default value of regex will change from True to False in a future version.
  data.doc = data.doc.str.replace("\s+", " ") # white space 삭제


In [ ]:
data.dropna(inplace=True)
data = data.sample(frac=1).reset_index(drop=True)
data.reset_index(drop=True, inplace=True)

In [ ]:
datar1 = pd.read_csv("/content/drive/MyDrive/대동여주도 리뷰.csv", encoding='utf-8-sig')
datar1.dropna(inplace=True)
datar1.reset_index(drop=True, inplace=True)

In [ ]:
datar2 = pd.read_csv("/content/drive/MyDrive/paiksool.csv", encoding='utf-8-sig')
datar2.dropna(inplace=True)
datar2.reset_index(drop=True, inplace=True)

In [ ]:
train_dataset =[]
for i in data.doc[:int(len(data)*0.9)]:
  train_dataset.append(i)

for i in datar1.text[:int(len(datar1.text)*0.9)]:
  train_dataset.append(i)

for i in datar2['내용'][:int(len(datar2['내용'])*0.9)]:
  train_dataset.append(i)


val_dataset = []
for i in data.doc[int(len(data)*0.9):]:
  val_dataset.append(i)

for i in datar1.text[int(len(datar1.text)*0.9):]:
  train_dataset.append(i)

for i in datar2['내용'][int(len(datar2['내용'])*0.9):]:
  train_dataset.append(i)

In [ ]:
import torch
from ratsnlp.nlpbook.generation import GenerationTrainArguments
args = GenerationTrainArguments(
    pretrained_model_name="skt/kogpt2-base-v2",
    downstream_model_dir="/content/drive/MyDrive/nlpbook/checkpoint-generation",
    max_seq_length=128,
    batch_size=32 if torch.cuda.is_available() else 4,
    tpu_cores=0 if torch.cuda.is_available() else 8,
    learning_rate=5e-5,
    epochs=5,
    seed=7,
    # batch_size=10,
    # cpu_workers = os.cpu_count()
)

In [ ]:
model = GPT2LMHeadModel.from_pretrained(
    args.pretrained_model_name
)

In [ ]:
tokenizer = PreTrainedTokenizerFast.from_pretrained("skt/kogpt2-base-v2",
  bos_token='<s>', eos_token='</s>', unk_token='<unk>',
  pad_token='<pad>', mask_token='<mask>') 


The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'GPT2Tokenizer'. 
The class this function is called from is 'PreTrainedTokenizerFast'.


In [ ]:
# tokenizer = PreTrainedTokenizerFast.from_pretrained(
#     args.pretrained_model_name,
#     eos_token="</s>",
# )

In [ ]:
from ratsnlp import nlpbook
nlpbook.set_seed(args)

set seed: 7


In [ ]:
nlpbook.set_logger(args)


INFO:ratsnlp:Training/evaluation parameters GenerationTrainArguments(pretrained_model_name='skt/kogpt2-base-v2', downstream_task_name='sentence-generation', downstream_corpus_name='nsmc', downstream_corpus_root_dir='/content/Korpora', downstream_model_dir='/content/drive/MyDrive/nlpbook/checkpoint-generation', max_seq_length=128, save_top_k=1, monitor='min val_loss', seed=7, overwrite_cache=False, force_download=False, test_mode=False, learning_rate=5e-05, epochs=5, batch_size=32, cpu_workers=2, fp16=False, tpu_cores=0)
INFO:ratsnlp:Training/evaluation parameters GenerationTrainArguments(pretrained_model_name='skt/kogpt2-base-v2', downstream_task_name='sentence-generation', downstream_corpus_name='nsmc', downstream_corpus_root_dir='/content/Korpora', downstream_model_dir='/content/drive/MyDrive/nlpbook/checkpoint-generation', max_seq_length=128, save_top_k=1, monitor='min val_loss', seed=7, overwrite_cache=False, force_download=False, test_mode=False, learning_rate=5e-05, epochs=5, bat

In [ ]:
import os
from ratsnlp.nlpbook.generation.corpus import GenerationExample
class NewsCorpus:

    def __init__(self):
        pass

    def get_examples(self, data_root_path, mode):
        data_fpath = os.path.join(data_root_path, f"{mode}.txt")
        lines = open(data_fpath, "r", encoding="utf-8").readlines()
        examples = []
        for (i, line) in enumerate(lines):
            if i == 0:
                continue
            text = line
            sentence = text 
            examples.append(GenerationExample(text=sentence))
        return examples

In [ ]:
# os.mkdir("./Korpora")
# os.mkdir("./Korpora/nsmc")
pd.DataFrame(train_dataset,columns=['doc']).doc.to_csv('./Korpora/nsmc/train.txt',encoding='utf-8-sig',index=False)

In [ ]:
pd.DataFrame(val_dataset,columns=['doc']).doc.to_csv('./Korpora/nsmc/test.txt',encoding='utf-8-sig',index=False)

In [ ]:
from ratsnlp.nlpbook.generation import GenerationDataset

corpus = NewsCorpus()

train_dataset = GenerationDataset(
    args=args,
    corpus=corpus,
    tokenizer=tokenizer,
    mode="train",
)

val_dataset = GenerationDataset(
    args=args,
    corpus=corpus,
    tokenizer=tokenizer,
    mode="test",
)

INFO:ratsnlp:Loading features from cached file /content/Korpora/nsmc/cached_train_PreTrainedTokenizerFast_128_nsmc_sentence-generation [took 1.789 s]
INFO:ratsnlp:Loading features from cached file /content/Korpora/nsmc/cached_train_PreTrainedTokenizerFast_128_nsmc_sentence-generation [took 1.789 s]
INFO:ratsnlp:Loading features from cached file /content/Korpora/nsmc/cached_test_PreTrainedTokenizerFast_128_nsmc_sentence-generation [took 0.087 s]
INFO:ratsnlp:Loading features from cached file /content/Korpora/nsmc/cached_test_PreTrainedTokenizerFast_128_nsmc_sentence-generation [took 0.087 s]


In [ ]:
from torch.utils.data import DataLoader, RandomSampler
train_dataloader = DataLoader(
    train_dataset,
    batch_size=args.batch_size,
    sampler=RandomSampler(train_dataset, replacement=False),
    collate_fn=nlpbook.data_collator,
    drop_last=False,
    num_workers=args.cpu_workers,
)

In [ ]:
from torch.utils.data import SequentialSampler

val_dataloader = DataLoader(
    val_dataset,
    batch_size=args.batch_size,
    sampler=SequentialSampler(val_dataset),
    collate_fn=nlpbook.data_collator,
    drop_last=False,
    num_workers=args.cpu_workers,
)

In [ ]:
from ratsnlp.nlpbook.generation import GenerationTask
task = GenerationTask(model, args)

In [ ]:
trainer = nlpbook.get_trainer(args)


INFO:pytorch_lightning.utilities.rank_zero:GPU available: True, used: True
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:IPU available: False, using: 0 IPUs
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs


In [ ]:
trainer.fit(
    task,
    train_dataloaders=train_dataloader,
    val_dataloaders=val_dataloader
)

INFO:pytorch_lightning.accelerators.gpu:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO:pytorch_lightning.callbacks.model_summary:
  | Name  | Type            | Params
------------------------------------------
0 | model | GPT2LMHeadModel | 125 M 
------------------------------------------
125 M     Trainable params
0         Non-trainable params
125 M     Total params
500.656   Total estimated model params size (MB)


Training: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

In [ ]:
prompt=" "
input_ids = tokenizer.encode(prompt, return_tensors="pt")
with torch.no_grad():
    generated_ids = model.generate(input_ids,
                           max_length=128,
                           do_sample=True,
                           temperature=0.95,
                           top_k=50,
                           top_p=0.95,
                           pad_token_id=tokenizer.pad_token_id,
                           eos_token_id=tokenizer.eos_token_id,
                           bos_token_id=tokenizer.bos_token_id,
                           repetition_penalty=2.0,       
                           use_cache=True
                          ) 


print(tokenizer.decode([el.item() for el in generated_ids[0]]))

텁지 않고 걸쭉한 질감을 가진 단호박 막걸리 스타일 좋아하시면 이강주는 필요 없을 것 같아요.
</s>


In [ ]:
prompt="느린마을 막걸리 "
input_ids = tokenizer.encode(prompt, return_tensors="pt")
with torch.no_grad():
    generated_ids = model.generate(input_ids,
                           max_length=128,
                           do_sample=True,
                           temperature=0.95,
                           top_k=50,
                           top_p=0.95,
                           pad_token_id=tokenizer.pad_token_id,
                           eos_token_id=tokenizer.eos_token_id,
                           bos_token_id=tokenizer.bos_token_id,
                           repetition_penalty=2.0,       
                           use_cache=True
                          ) 


print(tokenizer.decode([el.item() for el in generated_ids[0][:-1]]))

느린마을 막걸리 톡쏘는 탄산맛이 좀 덜하고 단맛도 적네요평소 즐기던 두통달시까지 먹어보려구요



In [ ]:
prompt="느린마을 막걸리 고소"
input_ids = tokenizer.encode(prompt, return_tensors="pt")
with torch.no_grad():
    generated_ids = model.generate(input_ids,
                           max_length=128,
                           do_sample=True,
                           temperature=0.95,
                           top_k=50,
                           top_p=0.95,
                           pad_token_id=tokenizer.pad_token_id,
                           eos_token_id=tokenizer.eos_token_id,
                           bos_token_id=tokenizer.bos_token_id,
                           repetition_penalty=2.0,       
                           use_cache=True
                          ) 


print(tokenizer.decode([el.item() for el in generated_ids[0][:-1]]))

느린마을 막걸리 고소하고 달달한 맛이 좋아요 선물하기에도 좋고 얼음 넣고 탄산 넣어도 텁하지 않아요



In [ ]:
prompt="느린마을 막걸리 담백"
input_ids = tokenizer.encode(prompt, return_tensors="pt")
with torch.no_grad():
    generated_ids = model.generate(input_ids,
                           max_length=128,
                           do_sample=True,
                           temperature=0.95,
                           top_k=50,
                           top_p=0.95,
                           pad_token_id=tokenizer.pad_token_id,
                           eos_token_id=tokenizer.eos_token_id,
                           bos_token_id=tokenizer.bos_token_id,
                           repetition_penalty=2.0,       
                           use_cache=True
                          ) 


print(tokenizer.decode([el.item() for el in generated_ids[0][:-1]]))

느린마을 막걸리 담백하고 달고 시원한 맛이 좋습니다 부드럽고 숙취가 없고 너무 좋아요



In [ ]:
prompt="느린마을 막걸리 우유"
input_ids = tokenizer.encode(prompt, return_tensors="pt")
with torch.no_grad():
    generated_ids = model.generate(input_ids,
                           max_length=128,
                           do_sample=True,
                           temperature=0.95,
                           top_k=50,
                           top_p=0.95,
                           pad_token_id=tokenizer.pad_token_id,
                           eos_token_id=tokenizer.eos_token_id,
                           bos_token_id=tokenizer.bos_token_id,
                           repetition_penalty=2.0,       
                           use_cache=True
                          ) 


print(tokenizer.decode([el.item() for el in generated_ids[0][:-1]]))

느린마을 막걸리 우유처럼 부드럽고 목구멍은 술냄새가 너무 나질 않아서 좋아요 다른분께도 선물해 드렸더니 맛있다고 그러네요



In [ ]:
prompt="느린마을 막걸리 달"
input_ids = tokenizer.encode(prompt, return_tensors="pt")
with torch.no_grad():
    generated_ids = model.generate(input_ids,
                           max_length=128,
                           do_sample=True,
                           temperature=0.95,
                           top_k=50,
                           top_p=0.95,
                           pad_token_id=tokenizer.pad_token_id,
                           eos_token_id=tokenizer.eos_token_id,
                           bos_token_id=tokenizer.bos_token_id,
                           repetition_penalty=2.0,       
                           use_cache=True
                          ) 


print(tokenizer.decode([el.item() for el in generated_ids[0][:-1]]))

느린마을 막걸리 달달해서 맛있어요 우유처럼 부드러운 느낌도 있어서 마시기 편해요

